In [1]:
import sys
sys.path.append('../../build')
sys.path.append('../')
import IPSModule as ips
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import lattpy as lp

In [ ]:
Q = 1
temperature = 1
eta = 0

rad = 10.0

epsilon = 1.0
sigma = 1.0

init_dis = 2 ** (1.0 / 6.0) * sigma
np.random.seed(42)  # for reproducibility
print(init_dis)

# generate the initial configuration
_dis = init_dis * 2 / np.sqrt(3)
latt = lp.Lattice.hexagonal(a = _dis)
latt.add_atom()
latt.add_connections()
# s = lp.Circle((0, 0), radius=6)
s = lp.Donut((0, 0), radius_outer = 6, radius_inner = 3)
latt.build(shape = s, primitive=True)
ax = latt.plot()
s.plot(ax)
plt.show()

num_particles = latt.data.positions.shape[0]
init_particles_positions = latt.data.positions
p = ips.NoseHooverSystem(num_particles, Q, temperature, eta)

for i in range(num_particles):
    for d in range(2):
        p.get_positions()[d][i] = init_particles_positions[i][d] / 2
        p.get_velocities()[d][i] = 0.0

pair_force_config = {
    "type": "LennardJones",
    "eps": epsilon,
    "sigma": sigma
}

confinement_config = {
    "type": "Radial",
    "rad": rad
}
simulator = ips.IPS_Simulator_NoseHoover(p)
simulator.init(pair_force_config, confinement_config)

In [ ]:
from utils import SimulationVisualizer, calculate_total_energy

num_frames = 50
num_step = 1000000
sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, rad = 10, draw_interval=num_step // num_frames, dt = 0.001, simulation_steps=10)

# energy_arr = []

# def get_energy_callback(simulator, p, step):
#     energy = calculate_total_energy(p.get_positions(), p.get_velocities(), epsilon=epsilon, sigma=sigma)
#     energy_arr.append(energy)

# sim_visualizer.add_callback(get_energy_callback)

html = sim_visualizer.run_animation(num_step)
display(html)

# sim_visualizer.run_animation_to_gif(num_step, "Nose_Hoover.gif")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

kin = sim_visualizer.energy_history['kinetic']

n = len(kin)
sampling_rate = 1  

fft_result = np.fft.fft(kin)
# use fftfreq to get the correct frequencies
freqs = np.fft.fftfreq(n, d=1/sampling_rate)
magnitude = np.abs(fft_result)

half_n = n // 2
freqs_positive = freqs[1:half_n]
magnitude_positive = magnitude[1:half_n]

peak_index = np.argmax(magnitude_positive)
dominant_freq = freqs_positive[peak_index]
dominant_period = 1 / dominant_freq if dominant_freq != 0 else None

print(f"main freq: {dominant_freq:.3f} Hz")
if dominant_period:
    print(f"main period: {dominant_period:.3f} s")

plt.figure(figsize=(10, 5))
plt.plot(freqs_positive, magnitude_positive, label='frequency spectrum')
plt.title('frequency spectrum for kinetic energy')
plt.xlabel('frequency (Hz)')
plt.ylabel('magnitude')
plt.grid(True)
plt.axvline(x=dominant_freq, color='r', linestyle='--', 
            label=f'main freq: {dominant_freq:.3f} Hz')
plt.legend()
plt.show()
